## CNN

CNN 卷积神经网络。一般模型的结构都包含：卷积层(conv), 池化层（pooling), 密集层(dense)。

对于 CNN 来讲，除了像 NN 那们我们需要学习到 Dense 上的参数，我们还需要学习到 conv 卷积层的参数。我们主要在卷积层上应用不同的 filter(kernel), 而且可能不止一个 filter。即在应用一个卷积层后我们的输出的通道会增加很多。


而在应用 filter 时，因为 kernel 的 size 会使得原输入的矩阵缩减，我们可能需要使用 padding 来操持原矩阵。

除了上面常见的三层外，Dropout 层也是不或缺的，我们可以用来防止过拟合。


In [1]:
import tensorflow as tf
from tensorflow import keras


In [7]:
(train_images, train_labels),(test_images,test_labels) = keras.datasets.mnist.load_data()

print(train_images.shape)
print(test_images.shape)

(60000, 28, 28)
(10000, 28, 28)


In [11]:
# mnist 数据每个样本只有一个通道, 我们需要把这个通常给 reshape 出来
# 并且对图片进行规一化

train_images = train_images.reshape((60000, 28,28,1))
test_images = test_images.reshape((10000, 28,28,1))

train_images = train_images / 255.0
test_images = test_images /255.0

### 创建模型

#### 先创建模型的卷积基础

In [12]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


#### 分析下卷积层的输入及其参数个数

1. 在输入层的卷积层我们在指定 `input_shape` 时不用指定其样本的 `batch size`, 即只是 `(28, 28, 1)` 而不是 `(60000, 28, 28, 1)`.
2. 卷积层 `Conv2D` 的 output shape 可以用 28 - 3 + 1 = 26 来计算。但是更复杂的公式是：

$$
O = \frac{(w - k + 2p)}{s} + 1
$$

其中，

    - O, output 的 shape
    - w, input 的 width
    - k, kernel 的 size
    - p, padding 的 size
    - s, stride 的 size


3. 池化层 `MaxPooling2D` 的 Output shape 可以用 26 / 2 = 13 来计算，即公式 $\frac{w}{k}$. 

各层的参数计算如下，对于一个卷积层输出的通道都会有一个偏置参数，而输入的每个通道都会有一个 kernel 的参数需要学习。对于池化层是没有参数需要学习的，所以参数为 0 。所以有如下表格：

|Layer (type)                 |Output Shape              |Param #   |
|-----------------------------|--------------------------|----------|
|conv2d (Conv2D)              |(None, 26, 26, 32)        |32 * (1 * (3 * 3) + 1) = 320       |
|max_pooling2d (MaxPooling2D) |(None, 13, 13, 32)        |0         |
|conv2d_1 (Conv2D)            |(None, 11, 11, 64)        |64 * (32 * (3 * 3) + 1) = 18496     |
|max_pooling2d_1 (MaxPooling2 |(None, 5, 5, 64)          |0         |
|conv2d_2 (Conv2D)            |(None, 3, 3, 64)          |64 * (64 * (3 * 3) + 1) = 36928    |



#### 接下来就是在原有的卷积层上添加正常有 NN 就行了


In [17]:
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

#### Compile and train the model

In [19]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 44s 736us/sample - loss: 0.1584 - accuracy: 0.9513
Epoch 2/5
60000/60000 [==============================] - 54s 897us/sample - loss: 0.0505 - accuracy: 0.9848
Epoch 3/5
60000/60000 [==============================] - 60s 997us/sample - loss: 0.0350 - accuracy: 0.9892- loss: 0.0350 - accuracy: 
Epoch 4/5
60000/60000 [==============================] - 57s 955us/sample - loss: 0.0266 - accuracy: 0.9919
Epoch 5/5
60000/60000 [==============================] - 52s 862us/sample - loss: 0.0205 - accuracy: 0.9936


In [21]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 2s 153us/sample - loss: 0.0312 - accuracy: 0.9902


In [22]:
print('loss :', test_loss)
print('accuarcy:', test_acc)

loss : 0.03122959039947309
accuarcy: 0.9902
